# Load Dataset

In [1]:
import pandas as pd
df_2class = pd.read_csv('acetylcholinesterase_bioactivity_data_2class_pIC50.csv')
df_3class = pd.read_csv('acetylcholinesterase_bioactivity_data_3class_pIC50.csv')

In [2]:
df_3class.head()

,Unnamed: 0,molecule_chembl_id,bioactivity_class,canonical_smiles,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,0,CHEMBL133897,active,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,312.325,2.8032,0.0,6.0,6.124939
1,1,CHEMBL336398,active,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,376.913,4.5546,0.0,5.0,7.000000
2,2,CHEMBL131588,inactive,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,426.851,5.3574,0.0,5.0,4.301030
3,3,CHEMBL130628,active,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,404.845,4.7069,0.0,5.0,6.522879
4,4,CHEMBL130478,active,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,346.334,3.0953,0.0,6.0,6.096910


In [3]:
df_3class_mw = df_3class[df_3class.MW > 1000]
len(df_3class_mw)
df_3class_mw.head()

,Unnamed: 0,molecule_chembl_id,bioactivity_class,canonical_smiles,MW,LogP,NumHDonors,NumHAcceptors,pIC50
2801,2801,CHEMBL3087677,inactive,COc1ccc2c(c1)c(CC(=O)NCCCNCCCNC1=CC(=O)C(NCCCN...,1046.066,6.76984,6.0,14.0,4.987163
2803,2803,CHEMBL3087675,intermediate,CC1=C(CC(=O)NCCCNCCCNC2=CC(=O)C(NCCCNCCCNC(=O)...,1043.316,7.64620,6.0,10.0,5.020452
2832,2832,CHEMBL3099500,intermediate,CN(CCCCNc1c2c(nc3c(NC(=O)CNC(=O)CC[C@H](NC(=O)...,1151.423,8.88370,7.0,13.0,3.946922
2930,2930,CHEMBL3220805,intermediate,C[C@@]12CC[C@H]3[C@]4(C)CCC[C@@]5(C)C(=O)OCC[N...,1075.655,14.52030,0.0,8.0,5.301030


In [4]:
columnNames = ["canonical_smiles", "pIC50", 'bioactivity_class']
df_smiles = pd.DataFrame(data=df_3class, columns=columnNames)
df_smiles = df_smiles.drop_duplicates(subset=['canonical_smiles', 'pIC50'])
df_smiles = df_smiles.reset_index(drop=True)
df_smiles

,canonical_smiles,pIC50,bioactivity_class
0,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,6.124939,active
1,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,7.000000,active
2,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,4.301030,inactive
3,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,6.522879,active
4,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,6.096910,active
...,...,...,...
3570,O=C(NCCCCCCNc1c2c(nc3ccccc13)CCCC2)c1cc2ccccc2o1,6.372634,intermediate
3571,O=C(NCCCCCCNc1c2c(nc3ccccc13)CCC2)c1cc2ccccc2o1,6.263603,inactive
3572,CCN(CCCCCCCCc1cccc(OC)c1)Cc1ccccc1OC,5.247952,intermediate
3573,CC(C)=CC[C@H]1C[C@@]2(CC=C(C)C)C(=O)O[C@@](CC=...,5.017729,intermediate


## Relabling Bioactivity Class
threshold values (IC50 < 1,000 nM = Actives while IC50 > 10,000 nM = Inactives, corresponding to pIC50 > 6 = Actives and pIC50 < 5 = Inactives)

In [5]:
active = df_smiles['pIC50'] > 6
inactive = df_smiles['pIC50'] < 5
df_smiles.loc[active, 'bioactivity_class'] = 'active'
df_smiles.loc[inactive, 'bioactivity_class'] = 'inactive'

In [6]:
print("inactive:",len(df_smiles[df_smiles.bioactivity_class=='inactive']))
print("intermediate:",len(df_smiles[df_smiles.bioactivity_class=='intermediate']))
print("active:",len(df_smiles[df_smiles.bioactivity_class=='active']))
print("total:", len(df_smiles))

inactive: 1379
intermediate: 217
active: 1979
total: 3575


## Remove Intermediate Class

In [7]:
intermediate_id = df_smiles[df_smiles.bioactivity_class=='intermediate'].index
df_smiles = df_smiles[df_smiles.bioactivity_class!='intermediate']
print(len(df_smiles))

3358


# Encode The Bioactivity Class

In [8]:
bioactivity_mapping = {'active': 1, 'inactive': 0}
df_smiles['bioactivity_code'] = df_smiles['bioactivity_class'].map(bioactivity_mapping)
df_smiles

C:\Users\alyss\AppData\Local\Temp\ipykernel_11372\3876861388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_smiles['bioactivity_code'] = df_smiles['bioactivity_class'].map(bioactivity_mapping)


,canonical_smiles,pIC50,bioactivity_class,bioactivity_code
0,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,6.124939,active,1
1,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,7.000000,active,1
2,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,4.301030,inactive,0
3,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,6.522879,active,1
4,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,6.096910,active,1
...,...,...,...,...
3567,COc1cccc2cc(C(=O)NCCCCCCNc3c4c(nc5ccccc35)CCCC...,7.612610,active,1
3569,O=C(NCCCCCCNc1c2c(nc3ccccc13)CCCCC2)c1cc2ccccc2o1,6.511449,active,1
3570,O=C(NCCCCCCNc1c2c(nc3ccccc13)CCCC2)c1cc2ccccc2o1,6.372634,active,1
3571,O=C(NCCCCCCNc1c2c(nc3ccccc13)CCC2)c1cc2ccccc2o1,6.263603,active,1


# Feature Extraction

## Generate Morgan Fingerprint with RDKit

In [9]:
from rdkit import Chem
from rdkit.Chem import AllChem

radius = 2
nBits = 1024
bit_list = []
morgan_fp = []
molecules = [Chem.MolFromSmiles(smiles) for smiles in df_smiles.canonical_smiles]
for mol in molecules:
    bit = {}
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol,useChirality=True, radius=radius, nBits = nBits, bitInfo=bit)
    morgan_fp.append(fingerprint)
    bit_list.append(bit)
morgan_fp_list = [list(fingerprint) for fingerprint in morgan_fp]

## Visualize Substucture from Morgan Fingerprint

In [ ]:
import numpy as np
mfpvector_1 = np.array(morgan_fp[6])
mfpvector_2 = np.array(morgan_fp[8])

print(np.nonzero(mfpvector_1))
print(np.nonzero(mfpvector_2))

In [ ]:
#try viisualize the substructure
from rdkit.Chem import Draw
from PIL import Image
import matplotlib.pyplot as plt
img1 = Draw.DrawMorganBit(molecules[6], 999, bit_list[6])
img2 = Draw.DrawMorganBit(molecules[8], 999, bit_list[8])

combined_img = Image.new('RGB', (img1.width + img2.width, max(img1.height, img2.height)))
combined_img.paste(img1, (0, 0))
combined_img.paste(img2, (img1.width, 0))

plt.imshow(combined_img)
plt.axis('off')
plt.show()

# Generate Fingerprints with PaDEL-Descriptor

In [ ]:
# load generated fingerprints
padel_fp = pd.read_csv('padel_fp_all.csv')
len(padel_fp)

In [ ]:
# Remove intermedite class fingerprints
padel_fp = padel_fp.drop(intermediate_id)
len(padel_fp)

# Split data for Cross Validation

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# features = padel_fp
features = pd.DataFrame(morgan_fp_list, columns=range(1024))
target = df_smiles.bioactivity_code

In [ ]:
len(features)

In [ ]:
from sklearn.model_selection import KFold, ShuffleSplit
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score,roc_auc_score, f1_score, mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score
from math import sqrt
import pickle
import matplotlib.pyplot as plt

ss = ShuffleSplit(n_splits=10, test_size=0.20, random_state=0)
# kfold = KFold(n_splits=5)
y_pred_list = []
importances_list = []

for idx, (train, val) in enumerate(ss.split(features)):
    # print(len(train), len(val))
    # print(len(features))
    X_train, X_val = features.iloc[train], features.iloc[val]
    y_train, y_val = target.iloc[train], target.iloc[val]
    model = RandomForestClassifier(n_estimators=100, random_state=41)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    y_pred_list.append(y_pred)

    file=open(f"classification/model_{idx+1}.pkl", "wb")
    pickle.dump(model, file)
    file.close()

    roc = roc_auc_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    q2 = cross_val_score(model, X_val, y_val, cv=5, scoring='r2').mean()
    print(f"auROC: {roc:.4f}, f1 score: {f1:.4f}, Q2: {q2:.4f}")
    
    importances = model.feature_importances_
    importances_list.append(importances)